In [1]:
from flim.experiments import utils, LIDSDataset, ToTensor
from flim.models.lcn import LCNCreator

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split

from torchvision import transforms

from torchsummary import summary
import torchmetrics as tm

from math import ceil, floor

import copy

from os import path

### Define a GPU/CPU device

In [2]:
if torch.cuda.is_available():
    device = torch.device(0)
else:
    device = 'cpu'

### Define the folder which contains your data

In [3]:
base_dir = path.join("..", "data", "corel")

### Train a feature extractor using FLIM

In [4]:
# load architecture
# in this example, the architecure specifies a feature extractor and a classifier
architecture = utils.load_architecture(path.join(base_dir, 'archs', 'arch-with-classifier.json'))

In [5]:
# load images and markers
images, markers = utils.load_images_and_markers(path.join(base_dir, 'images-and-markers'))

# get input shape
ncols     = images.shape[1]
nrows     = images.shape[2]
nchannels = images.shape[3]
nclasses  = 6

In [6]:
# build model and learn convolutional layers with FLIM
creator = LCNCreator(architecture, images=images, markers=markers, relabel_markers=True, device=device)
creator.build_model()

model = creator.get_LIDSConvNet()

Building conv1
Building activ1
Building pool1
Building conv2
Building activ2
Building pool2
Building linear1
Building activ3
Building drop1
Building linear2


In [7]:
#feeze feature extractor
for param in model.features.parameters():
    param.requires_grad = False

In [8]:
summary(model, (nchannels,nrows,ncols))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 32, 400, 400]        (2,400)
|    └─ReLU: 2-2                         [-1, 32, 400, 400]        --
|    └─MaxPool2d: 2-3                    [-1, 32, 200, 200]        --
|    └─Conv2d: 2-4                       [-1, 64, 200, 200]        (18,432)
|    └─ReLU: 2-5                         [-1, 64, 200, 200]        --
|    └─MaxPool2d: 2-6                    [-1, 64, 100, 100]        --
├─Sequential: 1                          []                        --
|    └─Flatten: 2-7                      [-1, 640000]              --
|    └─Linear: 2-8                       [-1, 512]                 327,680,512
|    └─ReLU: 2-9                         [-1, 512]                 --
|    └─Dropout: 2-10                     [-1, 512]                 --
|    └─Linear: 2-11                      [-1, 6]                 

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 32, 400, 400]        (2,400)
|    └─ReLU: 2-2                         [-1, 32, 400, 400]        --
|    └─MaxPool2d: 2-3                    [-1, 32, 200, 200]        --
|    └─Conv2d: 2-4                       [-1, 64, 200, 200]        (18,432)
|    └─ReLU: 2-5                         [-1, 64, 200, 200]        --
|    └─MaxPool2d: 2-6                    [-1, 64, 100, 100]        --
├─Sequential: 1                          []                        --
|    └─Flatten: 2-7                      [-1, 640000]              --
|    └─Linear: 2-8                       [-1, 512]                 327,680,512
|    └─ReLU: 2-9                         [-1, 512]                 --
|    └─Dropout: 2-10                     [-1, 512]                 --
|    └─Linear: 2-11                      [-1, 6]                 

### Train a MLP classifier

In [9]:
def get_metrics() -> dict:
    """
    Returns a dictionary of metrics to compute.
    """
    return {
        "acc": tm.Accuracy().to(device),
        "kappa": tm.CohenKappa(num_classes=6).to(device),
    }

def training_step(model: nn.Module,
                  batch_x: Tensor,
                  batch_y: Tensor,
                  optimizer: optim.Optimizer,
                  metrics: dict) -> Tensor:
    """
    One training step over one batch (batch_x, batch_y).

    Parameters
    ----------
    model   : nn.Module
        Model to train.
    batch_x : torch.Tensor
        Input batch.
    batch_y : torch.Tensor
        Target batch.
    optimizer : optim.Optimizer
        Optimizer to use.
    metrics : dict
        Metrics to compute.
    Returns
    -------
    loss : Tensor
        Loss computed on the batch.
    """
    
    output = model(batch_x)
    loss = F.cross_entropy(output, batch_y)
    nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


    for _, metric in metrics.items():
        metric(output.softmax(dim=1), batch_y)
    
    return loss

def evaluation_step(model: nn.Module, 
                    batch_x: torch.Tensor,
                    batch_y: torch.Tensor,
                    metrics: dict) -> Tensor:
    """
    One evaluation step over one batch (batch_x, batch_y).

    Parameters
    ----------
    model   : nn.Module
        Model to evaluate.
    batch_x : torch.Tensor
        Input batch.
    batch_y : torch.Tensor
        Target batch.
    metrics : dict
        Metrics to compute.
    Returns
    -------
    loss: Tensor
        Loss computed on the batch.
    """
    output = model(batch_x)
    loss = F.cross_entropy(output, batch_y)

    for _, metric in metrics.items():
        metric(output.softmax(dim=1), batch_y)
    
    return loss

In [10]:
# Define train set. It should be a folder with images named as 0000{label}_0000{imge_number}.{image_format}
# You can create a .txt file with the name of the images in the train set and another file with the name of the images in test set
trainset = LIDSDataset(path.join(base_dir, 'dataset'), path.join(base_dir, 'train.txt'), transform=ToTensor())
# split train set in train and validation set
# consider forcing splits to be stratified
trainset, valset = random_split(trainset, [ceil(len(trainset)*0.8), floor(len(trainset)*0.2)])

In [11]:
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, drop_last=False)
valloader = DataLoader(valset, batch_size=32, shuffle=False, drop_last=False)

In [12]:
lr      = 0.00001
w_decay = 0.1
epochs  = 30

# set feature extraction in evaluation mode  
model.features.eval()

# get the trainable parameters
params = [param for param in model.parameters() if param.requires_grad]

# optimizer
optimizer = optim.Adam(params, lr=lr, weight_decay=w_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, cooldown=3, factor=0.1, verbose=False)
# get evaluation metrics      
train_metrics = get_metrics()
val_metrics = get_metrics()

/home/afalcao/miniconda3/envs/FLIM/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [14]:
# save the model with the best validation kappa
best_model = None
best_kappa = -1

# training loop
for epoch in range(0, epochs):
    # training setp
    N = len(trainloader)
    train_loss = 0 
    for i, data in enumerate(trainloader):
        model.classifier.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        loss = training_step(model, inputs, labels, optimizer, train_metrics)
        train_loss += loss
        print(f"\r[training]\tepoch: {epoch}/{epochs-1}\titeration: {i}/{N-1}\tloss: {loss.item():.6f}", end="")

    train_loss = train_loss / N
    train_acc = train_metrics['acc'].compute()
    train_kappa = train_metrics['kappa'].compute()

    print(f"\r[training]\tepoch: {epoch}/{epochs-1}\tloss: {train_loss.item():.6f}\tacc: {train_acc:.6f}\tkappa: {train_kappa:.6f}")

    # validation step
    N = len(valloader)
    val_loss = 0
    for i, data in enumerate(valloader):
        with torch.no_grad():
            model.classifier.eval()
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            loss = evaluation_step(model, inputs, labels, val_metrics)
            val_loss += loss
            print(f"\r[validating]\tepoch: {epoch}/{epochs-1}\titeration: {i}/{N-1}\tloss: {loss.item():.6f}", end="")

    val_loss = val_loss / N
    val_acc = val_metrics["acc"].compute()
    val_kappa = val_metrics["kappa"].compute()

    print(f"\r[validating]\tepoch: {epoch}/{epochs-1}\tloss: {val_loss.item():.6f}\tacc: {val_acc:.6f}\tkappa: {val_kappa:.6f}")

    # save the model if it has the best validation kappa
    if epoch == 0 or val_kappa > best_kappa:
        best_kappa = val_kappa
        best_model = copy.deepcopy(model.state_dict())

    scheduler.step(val_loss)

RuntimeError: CUDA out of memory. Tried to allocate 1.22 GiB (GPU 0; 10.75 GiB total capacity; 7.50 GiB already allocated; 444.44 MiB free; 8.61 GiB reserved in total by PyTorch)

In [ ]:
# save best model weights
torch.save(best_model.state_dict(), "./model.pth")

### Evaluate your model

In [ ]:
# Define test set. It should be a folder with images named as 0000{label}_0000{imge_number}.{image_format}
# You can create a .txt file with the name of the images in the test set
testset = LIDSDataset(path.join(base_dir, 'dataset'), path.join(base_dir, 'test.txt'), transform=ToTensor())
testloader = DataLoader(testset, batch_size=32, shuffle=False, drop_last=False)

In [ ]:
# get evalutation metrics
test_metrics = get_metrics()

# set model in evaluation mode
model.load_state_dict(best_model)
model.eval()

# testing loop
N = len(testloader)
for i, data in enumerate(testloader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    loss = evaluation_step(model, inputs, labels, test_metrics)
    print(f"\r[testing]\titeration: {i}/{N-1}\tloss: {loss.item()}", end="")

test_acc = test_metrics["acc"].compute()
test_kappa = test_metrics["kappa"].compute()

print(f"\r[testing] acc: {test_acc:.6f}\tkappa: {test_kappa:.6f}")